In [54]:
import numpy as np
from scipy import integrate


In [88]:
X_t_test = np.array([[0,0,0.321,0]]).T
X = X_t_test.flatten()
print(X.shape)

(4,)


In [112]:
def swing_equation_ODE(X_t,t,K,P,I,gamma):
    """
    >The function takes as input:
    
    X_t: np.array of dimension 2Nx1, giving thetas and omegas at time t, it is of the form (theta_0,omega0,...,theta_N,omega_N)
    t: the time at which the integraion starts
    K: np.array of dimension NxN, representing the coupling strength between nodes
    P: np.array of dimension Nx1, giving the power at the nodes of the system
    I: np.array of dimension Nx1, giving the inertia constants at the nodes of the system
    gamma: np.array of dimension Nx1, giving the damping coefficients at the nodes of the system
    delta_t: float, representing timestep at which to advance the simulation

    >The function returns the expression of the swing equation ODE
    """
    K,P,I,gamma = get_parameters()
    N = len(P)
    dX_dt = np.zeros(2*N)
    for i in range(2*N):
        if i % 2 == 0:
            dX_dt[i] = X_t[i+1]
        else:
            dX_dt[i] = (1/I[i//2])*(P[i//2] - gamma[i//2]*X_t[i] + sum([K[i//2][j]*np.sin(X_t[2*j]-X_t[i-1]) for j in range(N)]))
    return dX_dt

def simulate_time_step(X_t,K,P,I,gamma,delta_t):
    """
    >The function takes as input:
    
    X_t: np.array of dimension 2Nx1 giving thetas and omegas at time t, it is of the form (theta_0,omega0,...,theta_N,omega_N)
    K: np.array of dimension NxN representing the coupling strength between nodes
    P: np.array of dimension Nx1 giving the power at the nodes of the system
    I: np.array of dimension Nx1 giving the inertia constants at the nodes of the system
    gamma: np.array of dimension Nx1 giving the damping coefficients at the nodes of the system
    
    >The function returns:
    X_t_plus_1 : np.array of dimension 2N+1 giving thetas and omegas at time t+1
    """
    # Obtaining the expression of the swing equation ODE
    X_t_plus_1 = integrate.odeint(swing_equation_ODE, X_t.flatten(), [0,delta_t],(K,P,I,gamma),full_output=True)[0]
    return X_t_plus_1


#Test case inputs
X_t_test = np.array([[0,0,0.321,0]]).T
K_test =  np.array([[-5,5],[5,-5]])
P_test = np.array([[1,1.5]]).T
I_test = np.array([[1,1]]).T
gamma_test = np.array([[1,1]]).T
delta_t_test = 0.1
def test_case_benchmark():
    """ The function returns a benchmark value to asses the simulate_time_step function based on the bus system
    in question 2 of ECE483 homework 2 """
    benchmark = X_t_plus_1 = simulate_time_step(X_t_test,K_test,P_test,I_test,gamma_test,delta_t_test)[0]
    return benchmark
def test_simulate_time_step():
    """
    We run the current version of the code with the inputs corresponding to the bus-system in question 2 of homework 2
    and assert the value using this example.
    """
    benchmark = test_case_benchmark()
    assert simulate_time_step(X_t_test,K_test,P_test,I_test,gamma_test,delta_t_test)[0].any() == benchmark.any()
    

In [113]:
test_case_benchmark()
# The simulation doesn't change anything as we expect from a static system ! 

array([0.   , 0.   , 0.321, 0.   ])

In [114]:
test_simulate_time_step()